CYCLE GAN TRAINING!

The purpose is transfer the stylel of impressionist paintings into paitnings of people in operating roomm

Hidden incentive: some friends are graduating and it'd a cool gift.... if it works

Had to learn about CycleGANs since we do not cover this in class, so here is what I learned at a basic concept:

We did learn about generator and discriminators and so this is basically that

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import time

# Constants
IMG_HEIGHT = 256
IMG_WIDTH = 256
OUTPUT_CHANNELS = 3
BATCH_SIZE = 2 # played around with 1-3 trying to weight memory vs computational intensity 
LAMBDA = 8  # cycle consistency loss is weighted 8 times higher than the adversarial loss in your overall loss function. 
#I wanted to have some creative freedom so i found that decreasing lamda gave me that becaue you get less faithful reconstructions, but too low and the model woudl collapse


##############################
# UP and DOWNSAMPLING BLOCKS #
##############################
# Downsapling block - to reduce spatial dimensions, extracts features
def downsample(filters, size, apply_norm=True):
    
    """
    filters - number of filters in the convolution
    size - kernel size
    apply_norm - whether to apply batch normalization
    """
    
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential() # from Keras, this is a linear neural network
    
    #convolution layer - extracts features from the image
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    """
    padding - to pad the input so output dimensions are determined solely by stride
    use_baias - set to False since using batch normalization
    """
    
    if apply_norm:
        result.add(layers.BatchNormalization()) #applying batch normalization: preventing extreme feature extraction + stabilizes training
    
    result.add(layers.LeakyReLU(0.2)) #using LeakyReLu
    
    return result

# Upsampling block - to increase the spatial dimensions, reconstructs the image
def upsample(filters, size, apply_dropout=False): #i tried dropout true, but it was looksing some details so i turned it off, but could be turned on for fun
    
    """
    filters- number of filters in the convolution
    size - kernel size
    apply_dropout - whether to apply dropout
    """
    initializer = tf.random_normal_initializer(0., 0.02)
    
    result = keras.Sequential()
    
    # deconvolution layer - similar architecture as convolution layer, but transposed
    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                      kernel_initializer=initializer, use_bias=False))
    
    result.add(layers.BatchNormalization())
    
    if apply_dropout:
        result.add(layers.Dropout(0.5)) #forces the model to learn more robust features from a highly compressed representation and not relying on single features
    
    result.add(layers.ReLU())
    
    return result

###############################
# GENERATOR AND DISCRIMINATOR #
###############################
# Generator model - here is where i had to learn abut the U architecture of the cycleGAN - this buis is the architecture of the generator
def build_generator():
    """Generator model"""
    inputs = layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, 3])
    
    #DOWNSAMPLING, FOLLOWED BY UPSAMPLING IN MIRROR AS THE U ARCHITECTURE FOR A CYCLEGAN : U ARCHITECTURE PRESERVES SPATIAL DIMENSIONS
    # Downsampling (output filter, kernel size, apply norm)
    down_stack = [
        downsample(64, 4, apply_norm=False),  # tensor shape output layer = (batch size (bs), height (128), width (128), number of filters (64))
        downsample(128, 4),  # (bs, 64, 64, 128)
        downsample(256, 4),  # (bs, 32, 32, 256)
        downsample(512, 4),  # (bs, 16, 16, 512)
        downsample(512, 4),  # (bs, 8, 8, 512)
        downsample(512, 4),  # (bs, 4, 4, 512)
    ]
    # above 512 layers is to create a bottle neck - which helps with feature saturatin and memory constraints 
    
    # Upsampling 
    up_stack = [
        upsample(512, 4, apply_dropout=True),  # (bs, 8, 8, 512)
        upsample(512, 4, apply_dropout=True),  # (bs, 16, 16, 512)
        upsample(256, 4),  # (bs, 32, 32, 256)
        upsample(128, 4),  # (bs, 64, 64, 128)
        upsample(64, 4),  # (bs, 128, 128, 64)
    ]
    #inverse as above to create the U
    
    
    #output layer
    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(OUTPUT_CHANNELS, 4,
                                 strides=2,
                                 padding='same',
                                 kernel_initializer=initializer,
                                 activation='tanh')  # (bs, 256, 256, 3)
    
    x = inputs
    
    # Downsampling through the model for x image
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x) #skip connections for the U architecture is something i learned about. skip connections help the model learn better by allowing gradients to flow through the network more easily
    
    skips = reversed(skips[:-1]) #reversed to match, but not the bottleneck one (-1)
    
    # Upsampling and establishing the skip connections
    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])
    
    x = last(x)
    
    return keras.Model(inputs=inputs, outputs=x)

# Discriminator model - this is the discriminator and it does not have a U architecture since we are trying to discriminate between the real and fake images
#i used a patchGAN discriminator which takes patches of the images to try to discriminate instad of the whole image. I read this was better 1) computation 2) in translation since we are interested in context and texture 
def build_discriminator():
    initializer = tf.random_normal_initializer(0., 0.02)
    
    inp = layers.Input(shape=[IMG_HEIGHT, IMG_WIDTH, 3], name='input_image')
    
    x = inp
    
    down1 = downsample(64, 4, False)(x)  # (bs, 128, 128, 64)
    down2 = downsample(128, 4)(down1)  # (bs, 64, 64, 128)
    down3 = downsample(256, 4)(down2)  # (bs, 32, 32, 256)
    
    zero_pad1 = layers.ZeroPadding2D()(down3)  # (bs, 34, 34, 256)
    conv = layers.Conv2D(512, 4, strides=1,
                        kernel_initializer=initializer,
                        use_bias=False)(zero_pad1)  # (bs, 31, 31, 512)
    
    batchnorm1 = layers.BatchNormalization()(conv)
    leaky_relu = layers.LeakyReLU()(batchnorm1)
    
    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)  # (bs, 33, 33, 512)
    last = layers.Conv2D(1, 4, strides=1,
                        kernel_initializer=initializer)(zero_pad2)  # (bs, 30, 30, 1)
    
    return keras.Model(inputs=inp, outputs=last)

##############################
# LOSS FUNCTIONS #
##############################
#how well the generator is fooling the discriminator
def generator_loss(generated_output):
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True) #using binarycross entropy, not by probability but how far away form 1
    return loss_object(tf.ones_like(generated_output), generated_output) 

#how well the discriminator can distinguish real from fake
def discriminator_loss(real_output, generated_output): 
    loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    real_loss = loss_object(tf.ones_like(real_output), real_output) #if real, close to 1; if fake close to 0
    generated_loss = loss_object(tf.zeros_like(generated_output), generated_output)
    total_loss = real_loss + generated_loss
    return total_loss * 0.5 #to prevent the discriminator from learning too quickly compared to the generator

 #Measures the L1 (absolute) difference between the original image and its reconstruction after a full cycle
 # to preserve the original paiting
def calc_cycle_loss(real_image, cycled_image):
    return tf.reduce_mean(tf.abs(real_image - cycled_image))

#Regularization term to ensure that the generator does not produce images that are too different from the original image
def identity_loss(real_image, same_image):
    return tf.reduce_mean(tf.abs(real_image - same_image)) * LAMBDA * 0.5

##############################
# GETTING DATA #
##############################
# GETTING THE PRE-PROCESSED DATA
def preprocess_image(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = (tf.cast(image, tf.float32) / 127.5) - 1 
    return image

#Grabs one image at the time (batch size = 1), shuffles them, and preprocesses them one by one
def create_dataset(image_paths, batch_size=1, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    
    dataset = dataset.batch(batch_size)
    return dataset

##############################
# SIMPLE LATENT SPACE EXPLORER#
##############################
# I have more functions for latent space exploration but this was an initial and simple one - for more go to visualization notebook
class LatentSpaceExplorer:
    def __init__(self, generator_g, generator_f):
        self.generator_g = generator_g  # painting -> OR
        self.generator_f = generator_f  # OR -> painting
    
    def show_samples(self, sample_x, sample_y): #just shows a sample - more later
        fake_y = self.generator_g(sample_x)
        fake_x = self.generator_f(sample_y)
        
        def denormalize(image):
            if isinstance(image, tf.Tensor):
                image = image.numpy()
            return (image + 1) * 0.5
        
        # Display images
        plt.figure(figsize=(12, 6))
        
        # Original
        plt.subplot(2, 2, 1)
        plt.imshow(denormalize(sample_x[0]))
        plt.title('Original Painting')
        plt.axis('off')
        
        plt.subplot(2, 2, 2)
        plt.imshow(denormalize(sample_y[0]))
        plt.title('Original OR Image')
        plt.axis('off')
        
        # Translations
        plt.subplot(2, 2, 3)
        plt.imshow(denormalize(fake_y[0]))
        plt.title('Painting → OR Image')
        plt.axis('off')
        
        plt.subplot(2, 2, 4)
        plt.imshow(denormalize(fake_x[0]))
        plt.title('OR Image → Painting')
        plt.axis('off')
        
        #Mixed / Translated
        plt.tight_layout()
        plt.suptitle('CycleGAN Translations')
        plt.show()
    
    def visualize_features(self, sample_image, generator, layer_idx=-4): #takes image, a generator, and a random layer - here i put 4
   
        layer = generator.layers[layer_idx]
        feature_model = keras.Model(inputs=generator.input, outputs=layer.output)
        features = feature_model(sample_image) #runs image through partial model to see intermediate 
        
        # Plot feature maps
        plt.figure(figsize=(12, 8))
        
        # Determine feature map dimensions
        num_features = min(16, features.shape[-1])
        rows = int(np.ceil(num_features / 4))
        
        for i in range(num_features):
            plt.subplot(rows, 4, i+1)
            
            # feature map
            feature_map = features[0, :, :, i]
            if isinstance(feature_map, tf.Tensor):
                feature_map = feature_map.numpy()
            
            # Normalize for visualization
            feature_map = (feature_map - np.min(feature_map)) / (np.max(feature_map) - np.min(feature_map) + 1e-7)
            
            plt.imshow(feature_map, cmap='viridis')
            plt.title(f'Feature {i+1}')
            plt.axis('off')
        
        plt.suptitle(f'Feature Maps from Layer: {layer.name}')
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.show()

##########################
# !!Training CycleGAN !! #
###########################
def train_cyclegan(train_x_dataset, train_y_dataset, epochs=200, patience=20):
    generator_g = build_generator()  # paintings to OR 
    generator_f = build_generator()  # OR to paintings
    discriminator_x = build_discriminator() #if an image is a real painting or fake
    discriminator_y = build_discriminator() #if an image is a real OR or fake
    
    # Optimizers
    generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    discriminator_x_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5) #discriminator learning rates lower to prevent overpowering
    discriminator_y_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)
    
    # Sample images for demonstration
    sample_x = next(iter(train_x_dataset))
    sample_y = next(iter(train_y_dataset))
    
    # For early stopping
    best_loss = float('inf')
    best_epoch = 0
    no_improvement = 0
    
    # Training loop
    for epoch in range(epochs):
        start = time.time()
        print(f"Epoch {epoch+1}/{epochs}")
        
        # Reset metrics for each epoch
        n = 0
        total_gen_g_loss = 0
        total_gen_f_loss = 0
        total_disc_x_loss = 0
        total_disc_y_loss = 0
        
        for x_batch, y_batch in tf.data.Dataset.zip((train_x_dataset, train_y_dataset)):
            losses = train_step(
                x_batch, y_batch,
                generator_g, generator_f,
                discriminator_x, discriminator_y,
                generator_g_optimizer, generator_f_optimizer,
                discriminator_x_optimizer, discriminator_y_optimizer
            )
            
            # Update loss totals
            total_gen_g_loss += losses['gen_g_loss']
            total_gen_f_loss += losses['gen_f_loss']
            total_disc_x_loss += losses['disc_x_loss']
            total_disc_y_loss += losses['disc_y_loss']
            n += 1
            
            # Print progress every 5 batches
            if n % 5 == 0:
                print(f"  Batch {n}: G loss: {losses['gen_g_loss']:.4f}, F loss: {losses['gen_f_loss']:.4f}")
        
        # Calculate average losses for the epoch
        avg_gen_g_loss = total_gen_g_loss/n
        avg_gen_f_loss = total_gen_f_loss/n
        avg_disc_x_loss = total_disc_x_loss/n
        avg_disc_y_loss = total_disc_y_loss/n
        
        # Combined generator loss for early stopping
        avg_gen_loss = (avg_gen_g_loss + avg_gen_f_loss) / 2
        
        # Early stopping check
        if avg_gen_loss < best_loss:
            best_loss = avg_gen_loss
            best_epoch = epoch
            no_improvement = 0
            
            # Save best models
            print(f"  New best model! Saving generators...")
            generator_g.save('best_generator_g.h5')
            generator_f.save('best_generator_f.h5')
        else:
            no_improvement += 1
            print(f"  No improvement for {no_improvement} epochs. Best epoch: {best_epoch+1}")
        
        # Epoch summary
        print(f"  Avg losses - G: {avg_gen_g_loss:.4f}, F: {avg_gen_f_loss:.4f}, D_X: {avg_disc_x_loss:.4f}, D_Y: {avg_disc_y_loss:.4f}")
        print(f"  Time: {time.time()-start:.2f} sec")
        
        # Generate example images every epoch
        generate_images(epoch, generator_g, generator_f, sample_x, sample_y)
        
        # Check if we should stop
        if no_improvement >= patience:
            print(f"Early stopping at epoch {epoch+1}. Best epoch was {best_epoch+1}.")
            
            # Load best models
            print("Loading best models...")
            generator_g = tf.keras.models.load_model('best_generator_g.h5')
            generator_f = tf.keras.models.load_model('best_generator_f.h5')
            break
    
    # Ensure we get back the best models, not the last ones
    if no_improvement > 0 and os.path.exists('best_generator_g.h5'):
        generator_g = tf.keras.models.load_model('best_generator_g.h5')
        generator_f = tf.keras.models.load_model('best_generator_f.h5')
    
    return generator_g, generator_f

def train_step(real_x, real_y, 
               generator_g, generator_f, 
               discriminator_x, discriminator_y,
               generator_g_optimizer, generator_f_optimizer,
               discriminator_x_optimizer, discriminator_y_optimizer):
    """Execute one training step"""
    with tf.GradientTape(persistent=True) as tape:
        # Generate fake images
        fake_y = generator_g(real_x, training=True)
        fake_x = generator_f(real_y, training=True)
        
        # Cycle consistency - taking fake back to original
        cycled_x = generator_f(fake_y, training=True)
        cycled_y = generator_g(fake_x, training=True)
        
        # Identity mapping - paintings remain paintings and OR images remain OR images
        same_x = generator_f(real_x, training=True)
        same_y = generator_g(real_y, training=True)
        
        # Discriminator outputs
        disc_real_x = discriminator_x(real_x, training=True)
        disc_real_y = discriminator_y(real_y, training=True)
        disc_fake_x = discriminator_x(fake_x, training=True)
        disc_fake_y = discriminator_y(fake_y, training=True)
        
        # Generator losses
        gen_g_loss = generator_loss(disc_fake_y)
        gen_f_loss = generator_loss(disc_fake_x)
        
        # Cycle consistency losses
        cycle_loss_x = calc_cycle_loss(real_x, cycled_x) * LAMBDA
        cycle_loss_y = calc_cycle_loss(real_y, cycled_y) * LAMBDA
        total_cycle_loss = cycle_loss_x + cycle_loss_y
        
        # Identity losses
        id_loss_x = identity_loss(real_x, same_x)
        id_loss_y = identity_loss(real_y, same_y)
        total_id_loss = id_loss_x + id_loss_y
        
        # Total generator losses
        total_gen_g_loss = gen_g_loss + total_cycle_loss + total_id_loss
        total_gen_f_loss = gen_f_loss + total_cycle_loss + total_id_loss
        
        # Discriminator losses
        disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
        disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
    
    # Calculate gradients- gradients are calculated for each model and applied using their respective optimizers
    generator_g_gradients = tape.gradient(total_gen_g_loss, generator_g.trainable_variables)
    generator_f_gradients = tape.gradient(total_gen_f_loss, generator_f.trainable_variables)
    discriminator_x_gradients = tape.gradient(disc_x_loss, discriminator_x.trainable_variables)
    discriminator_y_gradients = tape.gradient(disc_y_loss, discriminator_y.trainable_variables)
    
    # Apply gradients from above
    generator_g_optimizer.apply_gradients(zip(generator_g_gradients, generator_g.trainable_variables))
    generator_f_optimizer.apply_gradients(zip(generator_f_gradients, generator_f.trainable_variables))
    discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients, discriminator_x.trainable_variables))
    discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients, discriminator_y.trainable_variables))
    
    return {
        'gen_g_loss': gen_g_loss,
        'gen_f_loss': gen_f_loss,
        'disc_x_loss': disc_x_loss,
        'disc_y_loss': disc_y_loss
    }

def generate_images(epoch, generator_g, generator_f, test_input_x, test_input_y):
    if not os.path.exists('images'):
        os.makedirs('images')
    
    # Generate images
    prediction_y = generator_g(test_input_x)
    prediction_x = generator_f(test_input_y)
    
    # Function to denormalize images
    def denormalize(img):
        if isinstance(img, tf.Tensor):
            img = img.numpy()
        return (img + 1) * 0.5
    
    # Create figure
    plt.figure(figsize=(12, 12))
    
    display_list = [
        denormalize(test_input_x[0]),
        denormalize(prediction_y[0]),
        denormalize(test_input_y[0]),
        denormalize(prediction_x[0])
    ]
    
    title = ['Input Painting', 'Painting → OR Image', 'Input OR Image', 'OR Image → Painting']
    
    for i in range(4):
        plt.subplot(2, 2, i+1)
        plt.title(title[i])
        plt.imshow(display_list[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(f'images/epoch_{epoch+1}.png')
    plt.close()

# RUN CYCLE GAN + SIMPLE LATENT SPACE EXPLORATION
def run_cyclegan(train_paintings, train_or_images, epochs=5):
    train_paintings_ds = create_dataset(train_paintings, batch_size=1)
    train_or_images_ds = create_dataset(train_or_images, batch_size=1)
    
    # Train model
    print(f"Training CycleGAN for {epochs} epochs...")
    generator_g, generator_f = train_cyclegan(
        train_paintings_ds, train_or_images_ds, 
        epochs=epochs
    )
    
    # Create latent space explorer
    print("Exploring latent space:")
    explorer = LatentSpaceExplorer(generator_g, generator_f)
    
    # Show sample images
    sample_x = next(iter(train_paintings_ds))
    sample_y = next(iter(train_or_images_ds))
    explorer.show_samples(sample_x, sample_y)
    
    # Visualize features
    print("Generator features:")
    explorer.visualize_features(sample_x, generator_g)
    
    return generator_g, generator_f, explorer

# Execution
if __name__ == "__main__":
    # Define paths to the preprocessed datasets
    train_paintings = [os.path.join("preprocessed_paintings/train", f) for f in os.listdir("preprocessed_paintings/train") if f.endswith('.jpg')]
    test_paintings = [os.path.join("preprocessed_paintings/test", f) for f in os.listdir("preprocessed_paintings/test") if f.endswith('.jpg')]
    
    train_or_images = [os.path.join("preprocessed_or_images/train", f) for f in os.listdir("preprocessed_or_images/train") if f.endswith('.jpg')]
    test_or_images = [os.path.join("preprocessed_or_images/test", f) for f in os.listdir("preprocessed_or_images/test") if f.endswith('.jpg')]
    
    # Print found files to verify
    print(f"Found {len(train_paintings)} training paintings")
    print(f"Found {len(train_or_images)} training OR images")
    print(f"Found {len(test_paintings)} test paintings")
    print(f"Found {len(test_or_images)} test OR images")
    
    # Run CycleGAN
    generator_g, generator_f, explorer = run_cyclegan(
        train_paintings, 
        train_or_images, 
        epochs=2
    )

Found 4000 training paintings
Found 240 training OR images
Found 1000 test paintings
Found 54 test OR images
Training CycleGAN for 2 epochs...
Epoch 1/2


KeyboardInterrupt: 

In [ ]:
# Style Transfer Strength Control
# This code assumes you've already trained your CycleGAN model
# and have generator_g, generator_f available

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

def visualize_style_transfer_strength(generator, input_image, steps=8):
    """
    Visualize the effect of gradually increasing style transfer strength
    by blending between the original image and the fully translated image
    """
    # Ensure input is a batch
    if len(input_image.shape) == 3:
        input_image = tf.expand_dims(input_image, 0)
    
    # Generate the translated image
    translated_image = generator(input_image)
    
    # Create blends with different strengths
    blended_images = []
    for alpha in np.linspace(0, 1, steps):
        # Linear interpolation between original and translated
        blended = (1 - alpha) * input_image + alpha * translated_image
        blended_images.append(blended[0])
    
    # Plot the results
    plt.figure(figsize=(16, 4))
    
    # Function to convert from normalized [-1, 1] to display range [0, 1]
    def denormalize(img):
        if isinstance(img, tf.Tensor):
            img = img.numpy()
        return (img + 1) * 0.5
    
    # Display each blend
    for i, img in enumerate(blended_images):
        plt.subplot(1, steps, i+1)
        plt.imshow(denormalize(img))
        plt.title(f'Strength: {i/(steps-1):.2f}')
        plt.axis('off')
    
    plt.suptitle('Style Transfer Strength Control')
    plt.tight_layout()
    plt.show()
    
    return blended_images

# Helper function to create a dataset from image paths
def create_dataset(image_paths, batch_size=1, shuffle=True, img_height=256, img_width=256):
    """Create a TensorFlow dataset from image paths"""
    def preprocess_image(image_file):
        """Load and preprocess an image from file path"""
        image = tf.io.read_file(image_file)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [img_height, img_width])
        image = (tf.cast(image, tf.float32) / 127.5) - 1  # Normalize to [-1, 1]
        return image
        
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    
    dataset = dataset.batch(batch_size)
    return dataset

# Example usage: 
# Visualize painting to OR image translation with varying strength
def show_style_transfer_strength_demo(generator_g, generator_f, train_paintings_ds, train_or_images_ds):
    sample_painting = next(iter(train_paintings_ds))
    sample_or_image = next(iter(train_or_images_ds))
    
    print("Painting → OR Image with varying strength:")
    painting_to_or = visualize_style_transfer_strength(generator_g, sample_painting)
    
    print("\nOR Image → Painting with varying strength:")
    or_to_painting = visualize_style_transfer_strength(generator_f, sample_or_image)
    
    return painting_to_or, or_to_painting

# This section should be run after your GAN has been trained
# First, create the datasets
train_paintings = [os.path.join("preprocessed_paintings/train", f) for f in os.listdir("preprocessed_paintings/train") if f.endswith('.jpg')]
train_or_images = [os.path.join("preprocessed_or_images/train", f) for f in os.listdir("preprocessed_or_images/train") if f.endswith('.jpg')]

# Create the datasets
train_paintings_ds = create_dataset(train_paintings, batch_size=1)
train_or_images_ds = create_dataset(train_or_images, batch_size=1)

# Now run the visualization
# Assuming generator_g and generator_f are already defined from your training
painting_to_or, or_to_painting = show_style_transfer_strength_demo(generator_g, generator_f, train_paintings_ds, train_or_images_ds)

In [ ]:
# Progressive Generation Visualization
# This code shows how an image progresses through the generator network

def visualize_progressive_generation(generator, input_image):
    """
    Visualize how an image transforms through the main layers of the generator
    """
    # Ensure input is a batch
    if len(input_image.shape) == 3:
        input_image = tf.expand_dims(input_image, 0)
    
    # Create models for various depths of the generator
    intermediate_models = []
    
    # Find key layers to visualize (downsampling, bottleneck, and upsampling)
    key_indices = []
    bottleneck_found = False
    
    for i, layer in enumerate(generator.layers):
        # Skip the input layer
        if i == 0:
            continue
        
        # Look for significant layers like Conv2D, Conv2DTranspose, and Add
        layer_type = layer.__class__.__name__
        if layer_type in ['Conv2D', 'Conv2DTranspose', 'Add']:
            # Mark bottleneck (smallest spatial dimension)
            if not bottleneck_found and i > len(generator.layers) // 3:
                bottleneck_found = True
                key_indices.append(i)
            # Add a few key layers from downsampling and upsampling
            elif i % 4 == 0:  # Sample roughly every 4th significant layer
                key_indices.append(i)
    
    # Add the final output layer
    key_indices.append(len(generator.layers) - 1)
    
    # Limit to at most 8 layers to avoid overcrowding
    if len(key_indices) > 8:
        # Ensure we keep first, bottleneck, and last
        essential_indices = [key_indices[0], key_indices[len(key_indices)//2], key_indices[-1]]
        remaining = 5  # 8 - 3 essential indices
        
        # Select additional indices evenly distributed
        step = (len(key_indices) - 3) // (remaining + 1)
        selected_indices = []
        
        for i in range(1, remaining + 1):
            idx = i * step
            if idx < len(key_indices) - 1:  # Avoid the last index as it's already in essential_indices
                selected_indices.append(key_indices[idx])
        
        key_indices = sorted(essential_indices + selected_indices)
    
    # Create models for each key layer
    for idx in key_indices:
        layer = generator.layers[idx]
        intermediate_model = tf.keras.Model(inputs=generator.input, outputs=layer.output)
        intermediate_models.append((idx, layer.name, intermediate_model))
    
    # Generate intermediate outputs
    intermediate_outputs = []
    for idx, name, model in intermediate_models:
        output = model(input_image)
        intermediate_outputs.append((idx, name, output))
    
    # Plot the results
    num_outputs = len(intermediate_outputs)
    rows = int(np.ceil(num_outputs / 4))
    cols = min(num_outputs, 4)
    
    plt.figure(figsize=(16, 4 * rows))
    
    # Function to convert from normalized [-1, 1] to display range [0, 1]
    def denormalize(img):
        if isinstance(img, tf.Tensor):
            img = img.numpy()
        return (img + 1) * 0.5
    
    # Display input image
    plt.subplot(rows, cols, 1)
    plt.imshow(denormalize(input_image[0]))
    plt.title('Input Image')
    plt.axis('off')
    
    # Display intermediate outputs
    for i, (idx, name, output) in enumerate(intermediate_outputs[1:], start=2):
        plt.subplot(rows, cols, i)
        
        # Handle different output shapes
        if len(output.shape) == 4:
            # For feature maps, use the first 3 channels combined as RGB
            if output.shape[-1] > 3:
                # Take first 3 channels
                channels = output[0, :, :, :3]
                # Normalize each channel independently
                channels_normalized = tf.stack([
                    (channels[:, :, i] - tf.reduce_min(channels[:, :, i])) / 
                    (tf.reduce_max(channels[:, :, i]) - tf.reduce_min(channels[:, :, i]) + 1e-7)
                    for i in range(3)
                ], axis=-1)
                plt.imshow(channels_normalized)
            else:
                # If exactly 3 channels, treat as RGB
                plt.imshow(denormalize(output[0]))
        else:
            # For 2D outputs, use grayscale
            plt.imshow(output[0], cmap='gray')
        
        plt.title(f'Layer {idx}: {name}')
        plt.axis('off')
    
    plt.suptitle('Progressive Generation Through Generator Layers')
    plt.tight_layout()
    plt.show()
    
    return intermediate_outputs

# Function to showcase progressive generation for both generators
def show_progressive_generation_demo(generator_g, generator_f, train_paintings_ds, train_or_images_ds):
    # Get sample images
    sample_painting = next(iter(train_paintings_ds))
    sample_or_image = next(iter(train_or_images_ds))
    
    # Show progression for painting to OR image
    print("Progression: Painting → OR Image")
    painting_progression = visualize_progressive_generation(generator_g, sample_painting)
    
    # Show progression for OR image to painting
    print("\nProgression: OR Image → Painting")
    or_progression = visualize_progressive_generation(generator_f, sample_or_image)
    
    return painting_progression, or_progression

# Run this after training
# painting_progression, or_progression = show_progressive_generation_demo(generator_g, generator_f, train_paintings_ds, train_or_images_ds)